<a href="https://colab.research.google.com/github/suwatoh/Python-learning/blob/main/104_%E3%82%A4%E3%83%B3%E3%83%9D%E3%83%BC%E3%83%88%E3%82%B7%E3%82%B9%E3%83%86%E3%83%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

インポートシステム
==================

モジュールとパッケージ
----------------------

### モジュール ###

**モジュール**（module）は、Python の定義や文が実行される環境を表現するものであり、それ自体がオブジェクトである。

モジュールの `__name__` 属性の値は、そのモジュールの名前（文字列）となる。

トップレベルのコードが実行されるモジュールには自動的に `__main__` という名前が付けられる。ここで、トップレベルのコードとは、「Python インタープリター起動時に最初から実行され、プログラムに必要な他のすべてのモジュールをインポートするコード」という意味である。トップレベルのコードは、アプリケーションへの**エントリーポイント**（entry point）と呼ばれることもある。

Python インタープリターの対話モードでトップレベル環境を確認すると、次のようになる。

``` python
>>> __name__
'__main__'
```

`__main__` 以外のモジュールの名前には、インポート時に Python コードが保存されている場所のファイルシステム上の名前が使われる。ファイルなら、拡張子が `.py` でなければならず、そのファイルのベース名がモジュールの名前となる。このとき、ファイルシステムで使用できる文字と、Python 識別子に使用できる文字が異なることに注意する。Python 識別子に使用できない文字がファイルシステム上の名前に含まれる場合、インポート時に構文エラーが発生する。

`__name__` をチェックすれば各モジュールは自分がトップレベル環境で実行されているかどうかを知ることができる。このことから、モジュールがトップレベル環境で実行されている場合でのみ実行されるコードを次のように書ける:

``` python
if __name__ == '__main__':
    # Execute when the module is not initialized from an import statement.
    ...
```

モジュールにスクリプト用途のみのコードが含まれる場合には、そのスクリプト用コードを `if __name__ == '__main__'` の下のブロックに置くとよい。そうすれば、たとえばテスト目的で別のモジュールからインポートされる時に、スクリプト用コードが意図に反して実行されてしまうようなことがない。

`if __name__ == '__main__'` の下のブロックにあるコードはできるだけ少なくした方が、コードの分かりやすさや正確さにつながる。最もよくあるのが、プログラムの主要な処理を `main` 関数の中にカプセル化する方法である:

In [ ]:
def func(a, b):
    print(a + b)
    return a + b

def main():
    """フィボナッチ数列の第2項から第6項までを出力する"""
    x, y = 0, 1
    for i in range(2, 7):
        x, y = y, func(x, y)

if __name__ == "__main__":
    main()

1
2
3
5
8


このコードで、もし `main` 関数内のコードをカプセル化せず `if __name__ == '__main__'` の下に直接書いた場合、 `x`, `y` 変数はモジュール全体からグローバルにアクセスできてしまう。モジュール内の他の関数が意図せずローカル変数ではなくそのグローバル変数を使用してしまう可能性があるため、ミスにつながる。 `main` 関数を用意することでこの問題は解決できる。

### 通常のパッケージ ###

`__path__` 属性を持つモジュールは**パッケージ**（package）と呼ばれる。通常のパッケージでは、`__path__` 属性の値はパッケージが存在するディレクトリへのパスのリストとなる。Python モジュールがファイルシステム上にファイル（`.py` ファイル）として存在するのに対して、Python パッケージはファイルシステム上にディレクトリとして存在する。

ファイルシステム上のディレクトリ階層により、パッケージは階層構造を持つ。上位のパッケージを親パッケージ、下位のパッケージをサブパッケージと呼ぶ。パッケージの名前はドット `.` を含むことで階層構造を表現する。

In [ ]:
import importlib.metadata
print(f"{importlib.metadata.__path__ = }")
print(f"{importlib.metadata.__name__ = }")

importlib.metadata.__path__ = ['/usr/lib/python3.10/importlib/metadata']
importlib.metadata.__name__ = 'importlib.metadata'


通常のパッケージは、 `__init__.py` ファイルを含むディレクトリとする必要がある。通常のパッケージのインポート時には、この `__init__.py` ファイルが暗黙的に実行される（`__path__` 属性の初期化の後に実行される）。パッケージは単一の `__init__.py` ファイルだけを含むものであってもよい。`__init__.py` はただの空ファイルでも構わない。

たとえば、以下のようなファイルシステム配置は、3 つのサブパッケージを持つ最上位の `parent` パッケージを定義する:

``` text
parent/
    __init__.py
    one/
        __init__.py
        something.py
    two/
        __init__.py
    three/
        __init__.py
```

サブディレクトリ `one`、`two`、`three` が `__init__.py` ファイルを含むので、`parent.one`、`parent.two`、`parent.three` をパッケージとしてインポートすることができる。`parent.one` をインポートすると暗黙的に `parent/__init__.py` と `parent/one/__init__.py` が実行される。その後に `parent.two` もしくは `parent.three` をインポートすると、それぞれ `parent/two/__init__.py` や `parent/three/__init__.py` が実行される。

このパッケージ構成において、各パッケージの名前を確認するために、 `parent/__init__.py` と `parent/one/__init__.py` に以下のようなコードを書く。

`parent/__init__.py`:

``` python
from . import one
name = __name__
one_name = one.name
```

`parent/one/__init__.py`:

``` python
name = __name__
```

ここで、`parent` をサブディレクトリに持つプロジェクトルートで、Python インタープリターを起動すると、次のようにしてパッケージの名前を確認することができる。

``` python
>>> import parent
>>> print(parent.name)
parent
>>> print(parent.one_name)
parent.one
```

### モジュール検索パス ###

標準ライブラリの `sys` モジュールに含まれる `sys.path` 変数は、モジュールを検索するパスを示す文字列のリストである。`PYTHONPATH` 環境変数と、インストール時に指定したデフォルトパスで初期化される。

In [ ]:
import sys
sys.path

['/content',
 '/env/python',
 '/usr/lib/python310.zip',
 '/usr/lib/python3.10',
 '/usr/lib/python3.10/lib-dynload',
 '',
 '/usr/local/lib/python3.10/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.10/dist-packages/IPython/extensions',
 '/root/.ipython']

`sys.path` に含まれる空文字列 `''` はカレントディレクトリと解釈される。

`sys.path` の最初の要素は、Python インタープリターに渡すインターフェースオプションにより追加される。

``` shell
python [フラグ] [インターフェースオプション] [引数]
```

インターフェースオプションは次のとおり:

| オプション | 意味 | `sys.path[0]` |
|:---|:---|:---|
| `-c <command>` | command の Python 文を `__main__` モジュールとして実行する | カレントディレクトリ |
| `-m <module-name>` | 指定されたモジュール名のモジュールを探し、その内容を `__main__` モジュールとして実行する。引数はモジュール名なので、<br />拡張子（`.py`）を含めない。パッケージ名を与えた場合、`<pkg>.__main__.py` を `__main__` モジュールとして実行する | カレントディレクトリ |
| `<python-file>` | Python ファイル（拡張子 `.py`）を `__main__` モジュールとして実行する | ファイルを含むディレクトリ |
| `<directory>` | ディレクトリの中の `__main__.py` ファイルを `__main__` モジュールとして実行する | `<directory>` |
| `<zip-file>` | zip ファイル（拡張子 `.zip`）の中の `__main__.py` ファイルを `__main__` モジュールとして実行する | `<zip-file>` |
| `-` | 標準入力（`sys.stdin`）から読み込んだ Python コードを `__main__` モジュールとして実行する | カレントディレクトリ |

安全でない可能性のあるパスが `sys.path` の前に追加されることに注意する。 `-I`（隔離モード）フラグを指定して Python を起動する場合は、上記のようなパスが `sys.path` の前に追加されなくなる。隔離モードでは、ユーザーがインストールしたパッケージも、すべての `PYTHON*` 環境変数も無視される。

Python 3.11 で `-P` フラグが追加された。`-P` フラグを指定して Python を起動する場合は、上記のようなパスが `sys.path` の前に追加されなくなるが、ユーザーがインストールしたパッケージとすべての `PYTHON*` 環境変数は有効である。

`PYTHON*` 環境変数の例:

| 環境変数 | 機能 |
|:---|:---|
| `PYTHONHOME` | Python 標準ライブラリの場所を変更する |
| `PYTHONPATH` | モジュールファイルのデフォルトの検索パスを追加する |
| `PYTHONUSERBASE` | Python ユーザーインストールディレクトリの場所を変更する |
| `PYTHONSTARTUP` | この変数が読み込み可能なファイル名の場合、対話モードで最初のプロンプトが表示される前にそのファイルの Python コマンドが実行される |

### コマンドラインインターフェース ###

パッケージに含まれる `__main__.py` は、コマンドラインインターフェースをサポートするために用いられる。パッケージのコマンドラインインターフェースは、Python インタープリターに渡すインターフェースオプションにて `-m <module-name>` のようにパッケージが指定される形をとる。この場合、暗黙的にパッケージがインポートされた状態で `__main__.py` が実行される。`__init__.py` のインポート時実行がパッケージとしての実行であるのに対して、`__main__.py` の実行は（コマンドラインで `python __main__.py` と実行する場合と同等のトップレベル環境である） `__main__` モジュールとしての実行になる。

たとえば、カレントディレクトの直下に以下のようなファイルシステム配置を行うとする。

``` text
./
    my_package/
        __init__.py
        __main__.py
```

`__init__.py`:

``` python
print("__init__.py: パッケージとして実行された")

if __name__ == "__main__":
    print("__init__.py: __main__ モジュールとして実行された")
```

`__main__.py`:

``` python
print("__main__.py: ここから始まる")

if __name__ == "__main__":
    print("__main__.py: __main__ モジュールとして実行された")
```

コマンド `python -m my_package` を実行する場合、カレントディレクトの直下の `my_package` サブディレクトリは、`sys.path[0]` の値によりモジュール検索パスが通っている状態であることに注意する。実行結果は次のようになる:

``` shell
PS> python -m my_package
__init__.py: パッケージとして実行された
__main__.py: ここから始まる
__main__.py: __main__ モジュールとして実行された
```

`__init__.py` の if 文が実行されていないことに注意する。`__init__.py` は `my_package` パッケージとしてインポート時に実行されるので、`__name__` は `'my_package'` になっているからである。

なお、`-m` を使わないコマンド `python my_package` はインタープリターに直接ディレクトリを指定する形であり、これを実行する場合でも `__main__.py` が `__main__` モジュールとして実行される。ただし、暗黙的な `my_package` のインポートは行われないので、`__init__.py` は実行されない。実行結果は次のようになる:

``` shell
PS> python my_package
__main__.py: ここから始まる
__main__.py: __main__ モジュールとして実行された
```

### 名前空間パッケージ ###

実は、ディレクトリに `__init__.py` が無くてもディレクトリをパッケージとしてインポートすることは可能である。ただし、`__init__.py` が無い場合は、複数のパスに分散する同じ名前のディレクトリを 1 つのパッケージとして扱うようになる。このようなパッケージは通常のパッケージと区別して**名前空間パッケージ**と呼ぶ。

たとえば、以下のようなファイルシステム配置は、2 つのサブパッケージを持つ最上位の `parent` パッケージを定義する:

``` text
./
    spam/
        parent/
            one/
                __init__.py
    ham/
        parent/
            two/
                __init__.py
```

`parent` ディレクトリは `spam` と `ham` の 2 つのディレクトリに存在していて、どちらにも `__init__.py` が存在しないことに注意する。`spam` と `ham` を `sys.path` に含めれば、 `import parent` は名前空間パッケージをインポートすることになる。実際、`__path__` 属性を調べると、`spam` と `ham` のそれぞれの `parent` ディレクトリが認識されていることがわかる。

``` python
>>> import sys
>>> sys.path.append("spam")
>>> sys.path.append("ham")
>>> import parent
>>> parent.__path__
_NamespacePath(['C:\\Users\\User\\Desktop\\tmp\\spam\\parent', 'C:\\Users\\User\\Desktop\\tmp\\ham\\parent'])
```

インポート
----------

### import 文 ###

import 文の構文は次のとおり。

``` python
import <MODULE>[ as <IDENTIFIER>](, <MODULE>[ as <IDENTIFIER>], ...)
from <PACKAGE> import <SUBPACKAGE>[ as <IDENTIFIER>](, <SUBPACKAGE>[ as <IDENTIFIER>], ...)
from <MODULE> import <NAME>[ as <IDENTIFIER>](, <NAME>[ as <IDENTIFIER>], ...)
from <MODULE> import *
```

ここで、 `<MODULE>` はモジュールまたはパッケージの名前（識別子）であり、`<PACKAGE>` と `<SUBPACKAGE>` はそれぞれパッケージとサブパッケージの名前（識別子）である。また、 `<NAME>` は `<MODULE>` に含まれる関数、変数、クラスの名前（識別子）である。

`from <PACKAGE> import` と `from <MODULE> import` では、`import` に続くカンマで区切られた複数の節を明示的に `()` で括れば、次のように改行が可能である:

``` python
from <PACKAGE> import (
    <SUBPACKAGE> as <IDENTIFIER>,
    <SUBPACKAGE> as <IDENTIFIER>,
)
from <MODULE> import (
    <NAME> as <IDENTIFIER>,
    <NAME> as <IDENTIFIER>,
)
```

パッケージをインポートする際、 Python は `sys.path` 上のディレクトリを検索して、トップレベルのパッケージの入ったサブディレクトリを探す。

パッケージ内のモジュールでは、自身が含まれるサブパッケージをドット `.` で参照でき、親パッケージを二重ドット `..` で参照でき、親パッケージの親パッケージを `...` で参照できる（四重以上のドットも同様）。ドットを使ったインポートを**相対インポート**、そうでないインポートを**絶対インポート**と呼ぶ。

たとえば、`parent` パッケージの例では、`one` サブディレクトリの `__init__.py` の中で、次の import 文

``` python
from . import something
```

これは次の import 文と同等である。

``` python
from parent.one import something
```

また、次の import 文

``` python
from .. import two
```

これは次の import 文と同等である。

``` python
from parent import two
```

相対インポートはパッケージの階層構造に基づくので、パッケージではない（つまり `__path__` 属性を持たない）モジュールにおいては実行できない。たとえば、`python ./a.py` と実行する場合、`a.py` のコードは `__main__` モジュールとして実行されるが、この `__main__` はパッケージではないので、`from . import b` のようなコードはエラーとなる。

なお、`from <MODULE> import *` 文と、`__init__.py` の中で設定される `__all__` 変数について、次の公式チュートリアルを参照すること。

  * パッケージから * を import する: https://docs.python.org/ja/3/tutorial/modules.html#importing-from-a-package

PEP 8 は、`from <MODULE> import *` 文の使用を非推奨とする。これを使うと、どのクラスや関数がモジュールからインポートされたものかわかりにくく、またエディタの入力支援が受けられなくなる。

### 動的インポート ###

import 文ではモジュールを識別子で指定するほかないが、インポートするモジュールを識別子ではなく文字列で指定することは**動的インポート**（dynamic imports）と呼ばれる。動的インポートを使うと、たとえばプログラムの実行時に設定ファイルなどに基づいて選択的にモジュールをインポートするということができる。

動的インポートを行うには、標準ライブラリの `importlib` パッケージが提供する `import_module()` 関数を使用する。

``` python
importlib.import_module(name, package=None)
```

この関数は、動的インポートを行い、インポートしたモジュールを返す。`name` 引数は、インポートするモジュールを絶対または相対表現（たとえば `pkg.mod` または `..mod`）の文字列で指定する。`name` が相対表現で与えられたら、`package` 引数を、パッケージ名を解決するためのアンカーとなるパッケージの名前に設定する必要がある（たとえば `import_module('..mod', 'pkg.subpkg')` は `pkg.mod` をインポートする）。

In [ ]:
import importlib

m = "sys"
module = importlib.import_module(m)
module.path

['/content',
 '/env/python',
 '/usr/lib/python310.zip',
 '/usr/lib/python3.10',
 '/usr/lib/python3.10/lib-dynload',
 '',
 '/usr/local/lib/python3.10/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.10/dist-packages/IPython/extensions',
 '/root/.ipython']

### モジュールの再読み込み ###

インポートシステムの特徴として、**再インポートはモジュールを読み込まない**という点がある。この特徴により、複数のモジュールやパッケージを実行する場合に、重複する import 文を実行することになっても、同じモジュールの読み込みが行われることはない。モジュールの再読み込みには、次の関数を使う:

``` python
importlib.reload(module)
```

この関数は指定したモジュールを再読み込みする。モジュールを再読み込みすることによって、モジュールに加えた変更をプログラムを再起動せずに利用することができる。

モジュールがインポート済みであるか否かは、組み込み関数 `globals()` または `locals()` が返す辞書のキーを調べるとわかる。たとえば、`sys` モジュールがインポート済みであるなら再読み込みを行いたい場合は、次のように書く:

``` python
if "sys" in locals():
    import importlib
    importlib.reload(sys)  # noqa: F821
```

パス設定ファイル
----------------

Python を起動すると、標準ライブラリの `site` モジュールが自動的にインポートされる。`site` モジュールのインポート時に `site.main()` 関数が自動的に呼び出され、ユーザーによってインストールされたパッケージのファイルやメタデータが保存されるディレクトリを `sys.path` へ追加する。このディレクトリを site-packages ディレクトリと呼ぶ。Python インタープリターの `-S` オプションで `site` 自動インポートを禁止できる。

モジュール変数 `site.USER_SITE` で site-packages ディレクトリを確認できる。また、モジュール変数 `site.ENABLE_USER_SITE` が `True` の場合、 site-packages ディレクトリが `sys.path` に追加されている。

In [ ]:
import site
print(f"{site.USER_SITE=}")
print(f"{site.ENABLE_USER_SITE=}")

site.USER_SITE='/root/.local/lib/python3.10/site-packages'
site.ENABLE_USER_SITE=True


Python 環境の site-packages ディレクトリにある拡張子 `.pth` のファイルを**パス設定ファイル**と呼ぶ。パス設定ファイルは、1 行の Python コードからなり、`site.main()` 関数によって `sys.path` に項目を追加するために使用される。

開発中のパッケージを編集可能モード（`pip install` の `-e` オプション）でインストールする場合、 site-packages ディレクトリにはパッケージそのものではなく、パス設定ファイルが追加され、パス設定ファイルにより開発中パッケージのパスがモジュール探索パスに追加されるという仕組みになっている。

たとえば、カレントディレクトの直下に以下のようなファイルシステム配置を行うとする。

``` text
./
    greeting/
        __main__.py
        say.py
    pyproject.toml
```

`say.py`:

``` python
def say():
    print("Hello")
```

`__main__.py`:

``` python
from . import say

def main():
    say.say()

if __name__ == "__main__":
    main()
```

`pyproject.toml`:

``` ini
[project]
name = "greeting"
version = "1.0.0"

[project.scripts]
greet = "greeting.__main__:main"
```

カレントディレクトリに仮想環境を作成し、有効化しておく。この状態でも、`python -m greeting` が実行可能である:

``` shell
(.venv) PS> python -m greeting
Hello
```

カレントディレクトリを変えると、`greeting` はモジュール検索パスが通らないから `python -m greeting` の実行ではエラーが発生する。

`greeting` をパッケージとして編集可能モードでインストールすることにする。今回は、ローカルからインストールするだけで、PyPI などで配布する形式にビルドすることはしないから、`pyproject.toml` の中に `[build-system]` テーブルを作る必要はない。また、`[project]` テーブルで定義するメタデータは `name` と `version` で十分であるが、今回は実行形式も作成したいので `[project.scripts]` サブテーブルを記述している。

カレントディレクトリを戻し、プロジェクト（カレントディレクトリ）のパッケージを編集可能モードでインストールために `pip install -e .` コマンドを実行する。

``` shell
(.venv) PS> pip install -e .
  (省略)
Successfully built greeting
Installing collected packages: greeting
Successfully installed greeting-1.0.0
```

`greeting` という名前でパッケージがインストールされたことがわかる。`greeting` ディレクトリと同じ場所に `greeting.egg-info` というディレクトリが追加される。また、仮想環境の site-packages ディレクトリをみると、`greeting-1.0.0.dist-info` が追加されている。これらは、パッケージ管理に使用される。編集可能モードでは、パッケージ本体（`greeting` ディレクトリの中身）はインストールされず、その代わりに `__editable__.greeting-1.0.0.pth` と `__editable___greeting_1_0_0_finder.py` の 2 つのファイルが site-packages ディレクトリに追加される。これらにより、`greeting` ディレクトリがモジュール検索パスに追加される。

実際、カレントディレクトリを変更しても、`python -m greeting` を実行できる。また、`[project.scripts]` サブテーブルの記述により、`greet` 実行形式が作成される。

``` shell
(.venv) PS> greet
Hello
```

試しに `say.py` の `print()` 関数の引数を `'Good morning'` に書き換えてみると、（カレントディレクトリを変更しても）出力が変わる。

``` shell
(.venv) PS> python -m greeting
Good morning
(.venv) PS> greet
Good morning
```